# Рекомендация тарифов

In [2]:
# Все библиотеки, необходимые для выполнения проекта 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore') 

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.
Таким образом для каждого из 3214 клиентов мы знаем количество звонков, минут, сообщений и мегабайт-интернет трафика, а также каким тарифом пользуется клиент (если 0 то Смарт, если 1 то Ультра).
По условию данные в предобработке не нуждаются. Пропусков в них нет.

## Разбейте данные на выборки

In [27]:
# Разобьем исходные данные на обучающую (df_train), валидационную (df_val) и тестовую выборки (df_test) в соотношении 3:1:1
df_train, df_test = train_test_split(df, test_size=0.2, random_state=12345)
df_train, df_val = train_test_split(df_train, test_size=0.25, random_state=12345)
# Проверим получилось ли разделить данные в необходимом соотношении
df_train.shape[0], df_test.shape[0], df_val.shape[0]


(1928, 643, 643)

In [10]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_val = df_val.drop(['is_ultra'], axis=1)
target_val = df_val['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
features_test.shape, features_val.shape,

((643, 4), (643, 4))

In [11]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

## Исследуйте модели

### "Решающее дерево"

In [12]:
# Проверим влияние различных гиперпараметров на качество модели решающего дерева
dt = DecisionTreeClassifier(random_state = 12345)
parameters_dt = {'criterion': ['gini', 'entropy'], # критерий
                 'max_depth':range(1, 50), # глубина дерева
                 'min_samples_split': [0.1, 0.5, 0.7, 1.0], # минимальное количество выборок для разделения
                 'min_samples_leaf': range(1, 20), # минимальное количесто объектов в листе
                 'max_features':['auto', 'log2', 'sqrt', None]} # максимальное количество фич
search_dt = RandomizedSearchCV(dt, parameters_dt, cv=5, n_jobs = -1)
search_dt.fit(features_train, target_train)
best_dt = search_dt.best_estimator_
predict_dt = best_dt.predict(features_val)
print('Accuracy для Decision Tree на тренировочной выборке:', best_dt.score(features_train, target_train))
print('Accuracy для Decision Tree на валидационной выборке:', best_dt.score(features_val, target_val))

Accuracy для Decision Tree на тренировочной выборке: 0.8074679113185531
Accuracy для Decision Tree на валидационной выборке: 0.7776049766718507


### "Случайный лес"

In [13]:
# Проверим влияние различных гиперпараметров на качество модели случайного леса
rf = RandomForestClassifier(random_state = 12345)
parameters_rf = {'n_estimators': range(1, 200, 10), # количество деревьев 
                 'criterion': ['gini', 'entropy'], # критерий
                 'max_depth':range(1, 50, 1), # глубина дерева
                 'min_samples_split': [0.1, 0.5, 0.7, 1.0], # минимальное количество выборок для разделения
                 'min_samples_leaf': range(1, 10), # минимальное количесто объектов в листе
                 'max_features':['auto', 'log2', 'sqrt', None]} # максимальное количество фич
search_rf = RandomizedSearchCV(rf, parameters_rf, cv=5, n_jobs = -1)
search_rf.fit(features_train, target_train)
best_rf = search_rf.best_estimator_
predict_dt = best_rf.predict(features_val)
print('Accuracy для Random Forest на тренировочной выборке:', best_rf.score(features_train, target_train))
print('Accuracy для Random Forest на валидационной выборке:', best_rf.score(features_val, target_val))

Accuracy для Random Forest на тренировочной выборке: 0.809412679891093
Accuracy для Random Forest на валидационной выборке: 0.7776049766718507


### "Логистическая регрессия"

In [19]:
# Проверим влияние "параметров" на качество модели логистической регрессии
lr = LogisticRegression(random_state = 12345)
parameters_lr_1 = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 5, 10, 20, 50], # обратная сила регуляризации
                 'penalty':['l1'], # Регуляризация 
                 'solver':['liblinear', 'saga']} # функция потерь
parameters_lr_2 = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 5, 10, 20, 50], # обратная сила регуляризации
                 'penalty':['l2'], # Регуляризация 
                 'solver':['lbfgs', 'saga', 'sag', 'newton-cg']} # функция потерь

search_lr_1 = RandomizedSearchCV(lr, parameters_lr_1, cv=5, n_jobs = -1)
search_lr_1.fit(features_train, target_train)
best_lr_1 = search_lr_1.best_estimator_
predict_lr_1 = best_lr_1.predict(features_val)

search_lr_2 = RandomizedSearchCV(lr, parameters_lr_2, cv=5, n_jobs = -1)
search_lr_2.fit(features_train, target_train)
best_lr_2 = search_lr_2.best_estimator_
predict_lr_2 = best_lr_2.predict(features_val)

print('Accuracy для Logistic Regression (регуляризация l1) на тренировочной выборке :', best_lr_1.score(features_train, target_train))
print('Accuracy для Logistic Regression (регуляризация l1) на валидационной выборке:', best_lr_1.score(features_val, target_val))
print()
print('Accuracy для Logistic Regression (регуляризация l2) на тренировочной выборке:', best_lr_2.score(features_train, target_train))
print('Accuracy для Logistic Regression (регуляризация l2) на валидационной выборке:', best_lr_2.score(features_val, target_val))


Accuracy для Logistic Regression (регуляризация l1) на тренировочной выборке : 0.7464021781408012
Accuracy для Logistic Regression (регуляризация l1) на валидационной выборке: 0.7262830482115086

Accuracy для Logistic Regression (регуляризация l2) на тренировочной выборке: 0.7467911318553092
Accuracy для Logistic Regression (регуляризация l2) на валидационной выборке: 0.7262830482115086


In [20]:
data_for_score_val = [['decision tree', best_dt.score(features_val, target_val)],
                      ['random forest', best_rf.score(features_val, target_val)],
                      ['logistic regression (l1)', best_lr_1.score(features_val, target_val)], 
                      ['logistic regression (l2)', best_lr_2.score(features_val, target_val)]]
columns_for_score_val = ['model', 'accuracy']
table_for_score_val = pd.DataFrame(data = data_for_score_val, columns = columns_for_score_val)
table_for_score_val

,model,accuracy
0,decision tree,0.777605
1,random forest,0.777605
2,logistic regression (l1),0.726283
3,logistic regression (l2),0.726283


+ Вывод

Были исследованы три модели: как видно, наибольшей точностью модели обладает модель "случайного леса", что вполне логично (много деревьев по сравнению с одним решающим деревом).



## Проверьте модель на тестовой выборке

In [21]:
# Так как наибольшей точностью все-таки обладает модель случайного леса, то проанализируем её на тестовой выборке
accuracy_rf_test = best_rf.score(features_test, target_test)
print('Accuracy для Random Forest на тестовой выборке:', accuracy_rf_test)

Accuracy для Random Forest на тестовой выборке: 0.7931570762052877


+ Вывод

Качество модели случайного леса на тестовой выборке оказалось даже выше, чем необходимо

## (бонус) Проверьте модели на адекватность

In [22]:
# Посмотрим на соотношение классов в датасете 
df_smart = df[df['is_ultra'] == 0]
df_ultra = df[df['is_ultra'] == 1]
df_smart_len = df_smart.shape[0] 
df_ultra_len = df_ultra.shape[0]
ratio = df_smart_len / (df_smart_len + df_ultra_len)
print('Соотношение двух классов (тарифов "ультра" и "смарт"): ', ratio)
# Считается, что модель считается адекватной, если её точность выше, чем доля бОльшего класса в выборке,
# поэтому можно считать, что все модели, рассмотренные в шаге 4, могут быть адекватными

Соотношение двух классов (тарифов "ультра" и "смарт"):  0.693528313627878


Сравним настроенные модели с базовыми:

Первая модель - "решающее дерево".

In [23]:
dt_base = DecisionTreeClassifier(random_state=12345)
dt_base.fit(features_train, target_train)
accuracy_dt_base_test = dt_base.score(features_test, target_test)
print('Точность модели "решающее дерево":', accuracy_dt_base_test)

Точность модели "решающее дерево": 0.71850699844479


"Cлучайный лес"

In [24]:
rf_base = RandomForestClassifier(random_state=12345)
rf_base.fit(features_train, target_train)
accuracy_rf_base_test =  rf_base.score(features_test, target_test)
print('Точность модели "случайного леса":', accuracy_rf_base_test)

Точность модели "случайного леса": 0.776049766718507


"Логистическая регрессия"

In [25]:
lr_base = LogisticRegression(random_state=12345)
lr_base.fit(features_train, target_train)
accuracy_lr_base_test = lr_base.score(features_test, target_test)
print('Точность модели "логичестической регрессии":', accuracy_lr_base_test)

Точность модели "логичестической регрессии": 0.7573872472783826


In [26]:
data_base_create_ml = [['decision_tree', accuracy_dt_base_test, best_dt.score(features_test, target_test)], 
                    ['random_forest', accuracy_rf_base_test, best_rf.score(features_test, target_test)], 
                    ['logistic_regression', accuracy_lr_base_test, best_lr_1.score(features_test, target_test)]]
columns_base_create_ml = ['model', 'base','create']
table_base_create_ml = pd.DataFrame(data = data_base_create_ml, columns = columns_base_create_ml)
table_base_create_ml

,model,base,create
0,decision_tree,0.718507,0.790047
1,random_forest,0.776050,0.793157
2,logistic_regression,0.757387,0.760498


## Общий вывод

Были проанализированы модели решающего дерева, случайного леса и логистической регрессии на адекватность: получено, что все модели адекватны и имеют место для использования в данной задаче.
Примечательно то, что все три модели обладают accuracy > 0.75, и, таким образом, поставленная задача была выполнена. В дальнейшем нужно принять решение о выборе быстродействия модели - нужна ли в этой задачи максимальная точность, но при этом низкое быстродействие, или же наоборот - можно пренебречь несколькими процентами точности, но при этом модель будет работать гораздо быстрее.